In [174]:
import pytesseract
from PIL import Image, ImageOps
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import cv2
print(os.environ['TESSDATA_PREFIX'])

D:\Code\tessdata


In [175]:
def get_title(im):
    imarray = np.array(im)
    imarray_gray = cv2.cvtColor(imarray,cv2.COLOR_RGB2GRAY)
    _,  thresh = cv2.threshold(imarray_gray, 60, 255, cv2.THRESH_OTSU)
    title = thresh[0:int(imarray.shape[0]//8.9), int(imarray.shape[1]//4.3):int(imarray.shape[1]-1)]
    for i in range(title.shape[1]):
        cv2.floodFill(title, None, (i,title.shape[0]-1), 0)
        cv2.floodFill(title, None, (i,0), 0)
    kernel = np.ones((5,5), np.uint8)
    dilated = cv2.dilate(title, kernel, iterations = 2)
    cv2.floodFill(dilated, None, (0,0), 0)
    cv2.floodFill(dilated, None, (title.shape[1]-1,title.shape[0]-1), 0)
    title_cleaned = cv2.bitwise_and(title, dilated)
    # plt.imshow(title_cleaned)
    # plt.show()
    return pytesseract.image_to_string(Image.fromarray(title_cleaned))
    
def get_textbox(im):
    imarray = np.array(im)
    textbox = cv2.cvtColor(imarray[250:, :], cv2.COLOR_RGB2GRAY)
    _, thresh = cv2.threshold(textbox, 50, 255, cv2.THRESH_OTSU)
    cv2.floodFill(thresh, None, (0,0), 255)
    # plt.imshow(thresh)
    # plt.show()
    return pytesseract.image_to_string(Image.fromarray(thresh))

In [176]:
cards = []
for i in glob('./cards/*.png')[::-1]:
    im = Image.open(i)
    title = get_title(im).replace('\n', ' ')
    txtbox = get_textbox(im).replace('\n', ' ')
    card = {}
    card['name'] = title
    card['text'] = txtbox
    cards.append(card)

In [177]:
import json

# final = json.dumps(cards, indent=1)
with open("card_jsons/cloud_spirit_sword.json", 'w') as final:
    json.dump(cards, final)

